In [1]:
imagenet_path_drive = "/content/drive/MyDrive/Colab Notebooks/CV/project/imagenet"
path_drive = "/content/drive/MyDrive/Colab Notebooks/CV/project/"

In [2]:
!pip install pytorch-pretrained-biggan


     |████████████████████████████████| 132 kB 25.9 MB/s 
     |████████████████████████████████| 8.7 MB 46.4 MB/s 
     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 138 kB 12.1 MB/s 
     |████████████████████████████████| 127 kB 64.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def rgb_to_lab(srgb):

	srgb_pixels = torch.reshape(srgb, [-1, 3])

	linear_mask = (srgb_pixels <= 0.04045).type(torch.FloatTensor).cuda()
	exponential_mask = (srgb_pixels > 0.04045).type(torch.FloatTensor).cuda()
	rgb_pixels = (srgb_pixels / 12.92 * linear_mask) + (((srgb_pixels + 0.055) / 1.055) ** 2.4) * exponential_mask
	
	rgb_to_xyz = torch.tensor([
				#    X        Y          Z
				[0.412453, 0.212671, 0.019334], # R
				[0.357580, 0.715160, 0.119193], # G
				[0.180423, 0.072169, 0.950227], # B
			]).type(torch.FloatTensor).cuda()
	
	xyz_pixels = torch.mm(rgb_pixels, rgb_to_xyz)
	

	# XYZ to Lab
	xyz_normalized_pixels = torch.mul(xyz_pixels, torch.tensor([1/0.950456, 1.0, 1/1.088754]).type(torch.FloatTensor).cuda())

	epsilon = 6.0/29.0

	linear_mask = (xyz_normalized_pixels <= (epsilon**3)).type(torch.FloatTensor).cuda()

	exponential_mask = (xyz_normalized_pixels > (epsilon**3)).type(torch.FloatTensor).cuda()

	fxfyfz_pixels = (xyz_normalized_pixels / (3 * epsilon**2) + 4.0/29.0) * linear_mask + ((xyz_normalized_pixels+0.000001) ** (1.0/3.0)) * exponential_mask
	# convert to lab
	fxfyfz_to_lab = torch.tensor([
		#  l       a       b
		[  0.0,  500.0,    0.0], # fx
		[116.0, -500.0,  200.0], # fy
		[  0.0,    0.0, -200.0], # fz
	]).type(torch.FloatTensor).cuda()
	lab_pixels = torch.mm(fxfyfz_pixels, fxfyfz_to_lab) + torch.tensor([-16.0, 0.0, 0.0]).type(torch.FloatTensor).cuda()
	#return tf.reshape(lab_pixels, tf.shape(srgb))
	return torch.reshape(lab_pixels, srgb.shape)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import glob
import scipy
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from scipy.stats import truncnorm
from torch.nn.modules.batchnorm import LazyBatchNorm1d

import nltk
nltk.download('wordnet')

import matplotlib.pyplot as plt
import numpy as np
import copy
import pandas as pd
import PIL 
import pickle 
from tqdm import tqdm

from PIL import Image

import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)


# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")
    print("You may want to try to use the GPU in Google Colab by clicking in:")
    print("Runtime > Change Runtime type > Hardware accelerator > GPU.")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Using the GPU!


# Utility

In [5]:

def rgb_to_lab(srgb):

	srgb_pixels = torch.reshape(srgb, [-1, 3])

	linear_mask = (srgb_pixels <= 0.04045).type(torch.FloatTensor).cuda()
	exponential_mask = (srgb_pixels > 0.04045).type(torch.FloatTensor).cuda()
	rgb_pixels = (srgb_pixels / 12.92 * linear_mask) + (((srgb_pixels + 0.055) / 1.055) ** 2.4) * exponential_mask
	
	rgb_to_xyz = torch.tensor([
				#    X        Y          Z
				[0.412453, 0.212671, 0.019334], # R
				[0.357580, 0.715160, 0.119193], # G
				[0.180423, 0.072169, 0.950227], # B
			]).type(torch.FloatTensor).cuda()
	
	xyz_pixels = torch.mm(rgb_pixels, rgb_to_xyz)
	

	# XYZ to Lab
	xyz_normalized_pixels = torch.mul(xyz_pixels, torch.tensor([1/0.950456, 1.0, 1/1.088754]).type(torch.FloatTensor).cuda())

	epsilon = 6.0/29.0

	linear_mask = (xyz_normalized_pixels <= (epsilon**3)).type(torch.FloatTensor).cuda()

	exponential_mask = (xyz_normalized_pixels > (epsilon**3)).type(torch.FloatTensor).cuda()

	fxfyfz_pixels = (xyz_normalized_pixels / (3 * epsilon**2) + 4.0/29.0) * linear_mask + ((xyz_normalized_pixels+0.000001) ** (1.0/3.0)) * exponential_mask
	# convert to lab
	fxfyfz_to_lab = torch.tensor([
		#  l       a       b
		[  0.0,  500.0,    0.0], # fx
		[116.0, -500.0,  200.0], # fy
		[  0.0,    0.0, -200.0], # fz
	]).type(torch.FloatTensor).cuda()
	lab_pixels = torch.mm(fxfyfz_pixels, fxfyfz_to_lab) + torch.tensor([-16.0, 0.0, 0.0]).type(torch.FloatTensor).cuda()
	#return tf.reshape(lab_pixels, tf.shape(srgb))
	return torch.reshape(lab_pixels, srgb.shape)


In [6]:
def lab_to_rgb(lab):
		lab_pixels = torch.reshape(lab, [-1, 3])
		# convert to fxfyfz
		lab_to_fxfyfz = torch.tensor([
			#   fx      fy        fz
			[1/116.0, 1/116.0,  1/116.0], # l
			[1/500.0,     0.0,      0.0], # a
			[    0.0,     0.0, -1/200.0], # b
		]).type(torch.FloatTensor).cuda()
		fxfyfz_pixels = torch.mm(lab_pixels + torch.tensor([16.0, 0.0, 0.0]).type(torch.FloatTensor).cuda(), lab_to_fxfyfz)

		# convert to xyz
		epsilon = 6.0/29.0
		linear_mask = (fxfyfz_pixels <= epsilon).type(torch.FloatTensor).cuda()
		exponential_mask = (fxfyfz_pixels > epsilon).type(torch.FloatTensor).cuda()


		xyz_pixels = (3 * epsilon**2 * (fxfyfz_pixels - 4/29.0)) * linear_mask + ((fxfyfz_pixels+0.000001) ** 3) * exponential_mask

		# denormalize for D65 white point
		xyz_pixels = torch.mul(xyz_pixels, torch.tensor([0.950456, 1.0, 1.088754]).type(torch.FloatTensor).cuda())


		xyz_to_rgb = torch.tensor([
			#     r           g          b
			[ 3.2404542, -0.9692660,  0.0556434], # x
			[-1.5371385,  1.8760108, -0.2040259], # y
			[-0.4985314,  0.0415560,  1.0572252], # z
		]).type(torch.FloatTensor).cuda()

		rgb_pixels =  torch.mm(xyz_pixels, xyz_to_rgb)
		# avoid a slightly negative number messing up the conversion
		#clip
		rgb_pixels[rgb_pixels > 1] = 1
		rgb_pixels[rgb_pixels < 0] = 0

		linear_mask = (rgb_pixels <= 0.0031308).type(torch.FloatTensor).cuda()
		exponential_mask = (rgb_pixels > 0.0031308).type(torch.FloatTensor).cuda()
		srgb_pixels = (rgb_pixels * 12.92 * linear_mask) + (((rgb_pixels+0.000001) ** (1/2.4) * 1.055) - 0.055) * exponential_mask
	
		return torch.reshape(srgb_pixels, lab.shape)


#Dataset



In [7]:

%mkdir imagenet 
%cd imagenet 
!tar -xvf "/content/drive/MyDrive/Colab Notebooks/CV/project/ILSVRC2012_img_val.tar"
%cd /content



Streaming output truncated to the last 5000 lines.
ILSVRC2012_val_00049494.JPEG
ILSVRC2012_val_00048672.JPEG
ILSVRC2012_val_00048308.JPEG
ILSVRC2012_val_00047912.JPEG
ILSVRC2012_val_00045841.JPEG
ILSVRC2012_val_00044931.JPEG
ILSVRC2012_val_00044439.JPEG
ILSVRC2012_val_00043527.JPEG
ILSVRC2012_val_00042653.JPEG
ILSVRC2012_val_00042377.JPEG
ILSVRC2012_val_00041405.JPEG
ILSVRC2012_val_00040366.JPEG
ILSVRC2012_val_00035524.JPEG
ILSVRC2012_val_00033710.JPEG
ILSVRC2012_val_00031038.JPEG
ILSVRC2012_val_00030167.JPEG
ILSVRC2012_val_00024859.JPEG
ILSVRC2012_val_00024401.JPEG
ILSVRC2012_val_00023858.JPEG
ILSVRC2012_val_00023793.JPEG
ILSVRC2012_val_00022999.JPEG
ILSVRC2012_val_00021691.JPEG
ILSVRC2012_val_00021616.JPEG
ILSVRC2012_val_00020289.JPEG
ILSVRC2012_val_00019766.JPEG
ILSVRC2012_val_00019701.JPEG
ILSVRC2012_val_00017988.JPEG
ILSVRC2012_val_00017438.JPEG
ILSVRC2012_val_00015230.JPEG
ILSVRC2012_val_00014799.JPEG
ILSVRC2012_val_00014292.JPEG
ILSVRC2012_val_00012517.JPEG
ILSVRC2012_val_000119

In [8]:
class INColorDataset(Dataset):
    def __init__(self, subset, fig_size, split, transform=None):

        all_imgs = sorted(glob.glob("/content/imagenet/*"))
        labels = []
        with open(path_drive + "val_gt.txt") as f:
            for line in f:
                labels += [int(line)]

        self.map = self.get_map()

        if(subset == "5k"):
            all_imgs = all_imgs[:5000]
            labels = labels[:5000]
        elif(subset == "10k"):
            all_imgs = all_imgs[:10000]
            labels = labels[:10000]

        all_imgs_size = len(all_imgs)
        if(split == "train"):
            self.imgs = all_imgs[:int(0.6 * all_imgs_size)]
            self.labels = labels[:int(0.6 * all_imgs_size)]
            
        elif(split == "val"):
            self.imgs = all_imgs[int(0.6 * all_imgs_size) : int(0.8 * all_imgs_size)]
            self.labels = labels[int(0.6 * all_imgs_size) : int(0.8 * all_imgs_size)]
        else:
            self.imgs = all_imgs[int(0.8 * all_imgs_size) : ]
            self.labels = labels[int(0.8 * all_imgs_size) : ]
        
        self.transform = transform
        self.fig_size = fig_size

    def __len__(self):
        return len(self.imgs)

    def get_map(self):
        metadata = scipy.io.loadmat(path_drive + "meta.mat",struct_as_record=False)
        synsets = np.squeeze(metadata['synsets'])
        map = {s.ILSVRC2012_ID[0][0] : s.words[0].split(",")[0] for s in synsets[:1000]}
        return map

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = Image.open(img_path)

        img = img.convert("RGB")


        trans = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((self.fig_size, self.fig_size))                                   
        ])

        label = self.labels[idx]

        label = one_hot_from_names([self.map[label]],batch_size=1).argmax()


        img = trans(img)
        
        return img, label

In [9]:
def get_dataloader(subset, img_size, split, batch_size=16, num_workers=4):
    '''Use Pytorch torch.utils.data.DataLoader to load batched data'''
    
    dataset = INColorDataset(subset, img_size, split)

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(split == "train"),
        num_workers=num_workers,

    )
    return dataloader



# Model

In [10]:

class UNet(nn.Module):
    
    def unet_conv(self, ch_in, ch_out, is_leaky):
        """
        Construct a convolutional unit comprising of two conv layers
        followed by a batch normalisation layer and ReLU/Leaky ReLU.
        """
        if is_leaky:
            return nn.Sequential(
                nn.Conv2d(ch_in, ch_out, 3, padding=1),
                nn.BatchNorm2d(ch_out),
                nn.LeakyReLU(0.2),
                nn.Conv2d(ch_out, ch_out, 3, padding=1),
                nn.BatchNorm2d(ch_out),
                nn.LeakyReLU(0.2)
            )
        else:
            return nn.Sequential(
                nn.Conv2d(ch_in, ch_out, 3, padding=1),
                nn.BatchNorm2d(ch_out),
                nn.ReLU(),
                nn.Conv2d(ch_out, ch_out, 3, padding=1),
                nn.BatchNorm2d(ch_out),
                nn.ReLU()
            )
        
    def up(self, ch_in, ch_out):
        """
        Applies a 2D bilinear upsampling to the input image which scales
        the image 2x times, followed by a convolution with a 1x1 kernel. 
        """
        return nn.Sequential(
            nn.ConvTranspose2d(ch_in, ch_out, 3, 2, 1, 1), 
            nn.ReLU()
        )
    
    def __init__(self, is_leaky):
        """
        In the constructer, all the convolutional, upsampling and max pooling 
        units are instantiated and assigned as member variables. 
        """
        super(UNet, self).__init__()
        
        # First encoding layer
        self.conv1 = self.unet_conv(1, 64, is_leaky)
        # Second encoding layer
        self.conv2 = self.unet_conv(64, 128, is_leaky)
        # Third encoding layer
        self.conv3 = self.unet_conv(128, 256, is_leaky)
        # Fourth encoding layer
        self.conv4 = self.unet_conv(256, 512, is_leaky)
        # Fifth encoding layer
        self.conv5 = self.unet_conv(512, 1024, is_leaky)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2)
        
        # First Upsampling layer
        self.up1 = self.up(1024, 512)
        # Second Upsampling layer
        self.up2 = self.up(512, 256)
        # Third Upsampling layer
        self.up3 = self.up(256, 128)
        # Fourth Upsampling layer
        self.up4 = self.up(128, 64)
        
        # First decoding layer
        self.conv6 = self.unet_conv(1024, 512, False)
        # Second decoding layer
        self.conv7 = self.unet_conv(512, 256, False)
        # Third decoding layer
        self.conv8 = self.unet_conv(256, 128, False)
        # Fourth decoding layer
        self.conv9 = self.unet_conv(128, 64, False)
        
        # Last layer
        self.conv10 = nn.Conv2d(64, 2, 1)

    def forward(self, x):
        """ 
        An input tensor of a black and white image is accepted and
        passed through the U-Net model. A colored image in CieLAB color
        space is returned as the result. 
        """
        # Encoding path
        x1 = self.conv1(x)
        x2 = self.conv2(self.pool(x1))
        x3 = self.conv3(self.pool(x2))
        x4 = self.conv4(self.pool(x3))
        x5 = self.conv5(self.pool(x4))
        
        # Decoding path
        x = self.conv6(torch.cat((x4, self.up1(x5)), 1))
        x = self.conv7(torch.cat((x3, self.up2(x)), 1))
        x = self.conv8(torch.cat((x2, self.up3(x)), 1))
        x = self.conv9(torch.cat((x1, self.up4(x)), 1))
        x = self.conv10(x)
        m = nn.Tanh()
        x = m(x)
        
        return x




In [11]:
def evaluate(model, dataloader):
    running_loss = 0

    model.eval()
    with torch.no_grad():
        for imgs, _ in dataloader:
            imgs = imgs.to(device)
            imgs_LAB = rgb_to_lab(imgs)

            L = imgs[:, [0], :, :] / 100
            AB = imgs[:, 1:, :, :] / 100

            out = model(L)

            loss = nn.MSELoss()(out, AB)
            # statistics
            running_loss += loss.item() * imgs.size(0)

    print('Evaluation Loss: {:.4f}'.format(running_loss / len(dataloader.dataset)))
    return running_loss / len(dataloader.dataset)

In [15]:

'''
standard training loop
'''
def train_model(model, dataloaders, optimizer, scheduler, num_epochs=10):
    best_val_loss = 100000
    train_losses, val_losses = [], []


    for epoch in range(num_epochs):
        model.train()
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        running_loss = 0
        batches = tqdm(dataloaders["train"], total=len(dataloaders["train"]))

        for imgs, _ in batches:
            # check your input and target, these should be zero-mean one-std
            imgs = imgs.to(device)
            imgs_LAB = rgb_to_lab(imgs)
            
            L = imgs_LAB[:, [0], :, :] / 100
            AB = imgs_LAB[:, 1:, :, :] / 100

            
            # zero the parameter gradients
            optimizer.zero_grad()

            out = model(L)

            loss = nn.MSELoss()(out, AB)

            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * imgs.size(0)

            batches.set_description(
              "Current Loss ({:.4e}), running total ({:.4e}) ".format(
                   loss.item(), running_loss
              )
            ) 
        if(scheduler is not None):
            scheduler.step()
        
        epoch_loss = running_loss / len(dataloaders["train"].dataset)
        print('Epoch Loss: {:.4f}'.format(epoch_loss))
        train_losses.append(epoch_loss)

        # evaluate loss on val
        val_loss = evaluate(model, dataloaders["val"])
        val_losses.append(val_loss)

        if(val_loss < best_val_loss):
            print('\n Saving this model: \n')
            best_val_loss = val_loss
            torch.save(model.state_dict(), path_drive + "best_model_unet.pt")

    
    return model, train_losses, val_losses

In [16]:
dls = {split : get_dataloader("5k", 256, split) for split in ["train", "val", "test"]}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
'''
Only train encoder and classifier parameters
'''

unet = UNet(True).to(device)

optimizer = torch.optim.Adam(unet.parameters(), lr = 0.001, betas = (0.5, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.1)


In [17]:
unet, train_losses, val_losses = train_model(unet, dls, optimizer, None, num_epochs = 100)
pickle.dump(train_losses, open(path_drive + "unet_train_losses.pkl"))
pickle.dump(val_losses, open(path_drive + "unet_val_losses.pkl"))

Epoch 1/100
----------


  0%|          | 0/188 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Current Loss (9.6073e-03), running total (5.3166e+01) : 100%|██████████| 188/188 [03:24<00:00,  1.09s/it]

Epoch Loss: 0.0177


Evaluation Loss: 0.0004

 Saving this model: 

Epoch 2/100
----------


Current Loss (7.2368e-03), running total (3.4195e+01) : 100%|██████████| 188/188 [03:30<00:00,  1.12s/it]

...

Epoch Loss: 0.0108


Evaluation Loss: 0.0001

 Saving this model: 

Epoch 4/100
----------


Current Loss (1.0331e-02), running total (3.1737e+01) : 100%|██████████| 188/188 [03:31<00:00,  1.12s/it]

Epoch Loss: 0.0106


KeyboardInterrupt: ignored

# Evaluation

torch.Size([16, 1, 256, 256])


In [24]:


val_dataloader = get_dataloader('5k', 256, 'train')

out = None


with torch.no_grad():
    imgs, _ = iter(val_dataloader).next()
    imgs = imgs.to(device)
    imgs_LAB = rgb_to_lab(imgs)
            
    L = imgs_LAB[:, [0], :, :] / 100
    AB = imgs_LAB[:, 1:, :, :] / 100
    
    out = unet(L)


final_pred = torch.zeros(imgs.shape).to(device)
final_pred[:, [0], :, :] = L * 100
final_pred[:, 1:, :, :] = out * 100

final_pred = lab_to_rgb(final_pred)


for i in range(16):
    fig, ax = plt.subplots(1,2, figsize=(10,20))
    ax[0].imshow(imgs[i].detach().cpu().numpy().transpose(1,2,0))
    ax[1].imshow(final_pred[i].detach().cpu().numpy().transpose(1,2,0))


Output hidden; open in https://colab.research.google.com to view.